# Intro to using the PyShot Picker Tool
The PyShot picker tool is a tool for picking seismic events in a seismic signal. It is designed to be used with stream objects from the ObsPy library. 

First we import the PyShot package, as well as some other useful ones

In [ ]:
import pyshot
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Next we import the data we are interested in picking

Next we import the data we are interested in picking

In [ ]:
seismic_data = pyshot.load.load_segy.loadshots("./sample_data/")

This creates a dictionary of stream objects, where the keys are the names of the files and the values are the stream objects containing all the traces in that file.

We can now plot the data to see what we are working with using the ObsPy plot function

In [ ]:
seismic_data['33.su'].plot(type='section', scale=1, time_down=True, fillcolors=('blue', 'red'), color='black',
                size=(600, 800))

We can do some early processing of the data now to make picking easier and accurate.
We will demean and taper the data:

In [ ]:
for filename, stream in seismic_data.items():
    starttime = stream[0].stats.starttime
    stream.trim(starttime, starttime + 0.3) # This trims the data to the first 0.3 seconds
    for trace in stream:
        trace.detrend('linear')
        trace.detrend('demean')
        # trace.filter('bandpass', freqmin=10, freqmax=500, corners=4, zerophase=True)
        trace.data = np.require(trace.data, dtype=np.float32) # This fixes some issues with the data type

In [ ]:
# Now produce a spectrogram of the data to see what we are working with
#import numpy as np
#from scipy import signal
#
#stream = seismic_data['33.su']
#fig, axes = plt.subplots(nrows=len(stream), ncols=1, figsize=(10, 5*len(stream)))
#
#for i, tr in enumerate(stream):
#    # Use raw data without AGC
#    f, t, Sxx = signal.spectrogram(tr.data, fs=tr.stats.sampling_rate, nperseg=256, noverlap=128)
#    
#    # Apply stronger logarithmic scaling to enhance faint signals even more
#    Sxx_log = np.log1p(Sxx * 1000)  # Multiply by 1000 before log to boost faint signals
#    
#    # Normalize each time slice to further emphasize faint signals at later times
#    Sxx_norm = Sxx_log / np.max(Sxx_log, axis=0)
#    
#    # Use a colormap that emphasizes high-frequency content and faint signals
#    im = axes[i].pcolormesh(t, f, Sxx_norm, shading='gouraud', cmap='viridis', vmin=0, vmax=1)
#    axes[i].set_ylabel('Frequency [Hz]')
#    axes[i].set_title(f'Trace {i+1}')
#    
#    # Add colorbar to show intensity scale
#    plt.colorbar(im, ax=axes[i], label='Normalized Log(Power)')
#    
#    # Adjust y-axis to focus on higher frequencies where faint signals might be
#    axes[i].set_ylim(0, 500)  # Adjust this value as needed
#
#axes[-1].set_xlabel('Time [sec]')
#for ax in axes:
#    ax.set_ylim(0, 500)  # Increase y-axis limit to 1000 Hz to show higher frequencies
#plt.tight_layout()
#plt.show()

Now let's try and pick the data using the PyShot picker tool. The UI may behave differently depending on your operating system, but it should pop up in a separate window. 

The shortcut keys are as follows:

'x' - toggles picking mode on and off

'm' - decreases the amplitude scale of the plot

'n' - increases the amplitude scale of the plot

'j' - decreases the clip value of the plot slightly

'k' - increases the clip value of the plot slightly

'u' - increases the clip value of the plot by a factor of 2

'i' - decreases the clip value of the plot by a factor of 2

'v' - saves all picks to 2 csv files: one for primary arrivals, and one for secondary arrivals

'c' - saves just primary arrivals to a csv file. This is useful for firn density analysis when some direct arrivals are not accompanied by reflected arrivals, and you want to save direct-reflected arrival pick pairs separately from a larger set of direct arrival picks for firn density analysis.


In [ ]:
mpl.use('QtAgg')

# Pick the data
pyshot.pick.picker.Pick(seismic_data['33.su'], filename="33.su", outfile="pick_outputs/33_picks.csv")